In [ ]:
# importamos la librería simpy
import simpy
import numpy as np
import statistics as stats

En primer lugar vamos a hacer el set up básico de un entorno para la simulación que incluye:

In [ ]:
# 1) SEMILLA Y ENTORNO =========================
# 👉 Environment: el "reloj" y la agenda de eventos.
seed = 42
rng = np.random.default_rng(seed)
env = simpy.Environment()

# 2) RECURSOS Y ESTRUCTURAS ====================
# 👉 Resource: capacidad limitada (agentes que atienden pedidos).
#    capacity=2 significa que pueden atenderse 2 pedidos a la vez.
agents = simpy.Resource(env, capacity=2)

# 👉 Container: inventario continuo (unidades disponibles).
#    get(q) saca y espera si no hay; put(q) repone.
inventory = simpy.Container(env, init=3, capacity=999)

# 👉 Store: cola de objetos (aquí "pedidos").
#    put(obj) encola, get() desenfila.
inbox = simpy.Store(env, capacity=999)

Ahora definiremos los parámetro del modelo, lo cual es lo que da la utilidad a la hora de estudiar como se comportan fenómenos que queremos estudiar. Los parámetros se pueden modificar experimentalmente para estudiar el comportamiento del modelo en diferentes ocasiones.

Siéntete libre de modificar los parámetros tanto como quieras para estudiar el comportamiento del modelo.

In [ ]:
# 3) PARÁMETROS DEL MODELO =====================
MEAN_INTERARRIVAL = 1.5     # tiempo medio entre llegadas
MEAN_SERVICE = 1.0          # tiempo de servicio medio
WARMUP = 0.0                # para demo, sin warmup
HORIZON = 25                # tiempo total de simulación
RESTOCK_EVERY = 5           # cada cuánto reabastecemos
RESTOCK_QTY = 3
SETUP_TIME = 0.2            # tiempo de preparación (lo usamos en AllOf)
MEAN_PACIENCE = 1.0         # paciencia promedio de un pedido (cuanto tiempo está dispuesto a esperar
                            # antes de irse sin ser atendido)

Consiguientemente añadimos las variables (métricas) en donde vamos a guardar los resultados de la simulación. En este caso Tenemos pocas métricas, pero estas se pueden ajustar a la necesidad de cada proyecto para obtener los datos necesarios.

In [ ]:
# Métricas simples
served, abandoned = 0, 0
wait_times, sojourn_times = [], []